In [ ]:
!nvidia-smi
!nvcc --version

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

In [ ]:
%pip install ultralytics supervision roboflow
import ultralytics
ultralytics.checks()

In [ ]:
!yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://www.thewallstreetexperience.com/wp-content/uploads/2021/12/pexels-helena-lopes-1389339-1.jpg' save=True

In [ ]:
import os
import random
import shutil
from pathlib import Path

# Set paths
dataset_path = "/home/kobe/Desktop/Peppernet/Data/"
images_dir = os.path.join(dataset_path, "images")
labels_dir = os.path.join(dataset_path, "labels")
classes_file = os.path.join(dataset_path, "classes.txt")
output_yaml = os.path.join(dataset_path, "data.yaml")

# Split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Collect all image files
image_files = sorted(Path(images_dir).glob("**/*.jpg"))  # Adjust extension if necessary
total_files = len(image_files)

# Shuffle and split
random.seed(42)  # For reproducibility
random.shuffle(image_files)
train_files = image_files[:int(train_ratio * total_files)]
val_files = image_files[int(train_ratio * total_files):int((train_ratio + val_ratio) * total_files)]
test_files = image_files[int((train_ratio + val_ratio) * total_files):]

# Create directories for split datasets
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(images_dir, split), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, split), exist_ok=True)

# Move files
def move_files(files, split):
    for file in files:
        image_dest = os.path.join(images_dir, split, file.name)
        label_src = os.path.join(labels_dir, file.stem + ".txt")
        label_dest = os.path.join(labels_dir, split, file.stem + ".txt")

        shutil.move(file, image_dest)
        if os.path.exists(label_src):
            shutil.move(label_src, label_dest)

move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

# Read class names from classes.txt
if os.path.exists(classes_file):
    with open(classes_file, "r") as f:
        class_names = [line.strip() for line in f.readlines()]
else:
    raise FileNotFoundError(f"Classes file not found at {classes_file}")

# Generate data.yaml
data_yaml_content = f"""
train: {os.path.join(images_dir, 'train')}
val: {os.path.join(images_dir, 'val')}
test: {os.path.join(images_dir, 'test')}
nc: {len(class_names)}
names: {class_names}
"""

with open(output_yaml, "w") as f:
    f.write(data_yaml_content)

print(f"Data split completed and 'data.yaml' created at {output_yaml}")


In [ ]:
from ultralytics import YOLO
import torch

# Limit GPU memory usage
torch.cuda.set_per_process_memory_fraction(1.)  # Adjust to limit memory to 90%

# Load the pre-trained YOLOv8 model
model = YOLO('/home/kobe/Desktop/Peppernet/yolov8n.pt')

# Train the model
model.share_memory()
results = model.train(
    data='/home/kobe/Desktop/Peppernet/Data/data.yaml',
    epochs=3,
    imgsz=640,
    batch=32,
    device='cuda',           # Use GPU if available
    workers=2,
    augment=True,
    #patience=10,
    verbose=True
)

In [1]:
from ultralytics import YOLO
import torch
model = YOLO('/home/kobe/Desktop/JUPYTER-ML/runs/detect/train/weights/best.pt')

In [ ]:
# Validate the trained model
metrics = model.val(data='/home/kobe/Desktop/JUPYTER-ML/Military-Vehicle-Recognition-6/data.yaml')
print(metrics)  # Displays metrics like mAP, precision, recall, etc.

In [3]:
# Export the trained model
model = YOLO('/home/kobe/Desktop/Peppernet/runs/detect/train4/weights/best.pt')
model.export(format='onnx', opset=12)

Ultralytics 8.3.36 🚀 Python-3.12.7 torch-2.5.1+cu121 CPU (Intel Core(TM) i7-4770 3.40GHz)
Model summary (fused): 168 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/home/kobe/Desktop/Peppernet/runs/detect/train4/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 15, 8400) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.41...
ONNX: export success ✅ 0.7s, saved as '/home/kobe/Desktop/Peppernet/runs/detect/train4/weights/best.onnx' (11.7 MB)

Export complete (1.0s)
Results saved to /home/kobe/Desktop/Peppernet/runs/detect/train4/weights
Predict:         yolo predict task=detect model=/home/kobe/Desktop/Peppernet/runs/detect/train4/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/home/kobe/Desktop/Peppernet/runs/detect/train4/weights/best.onnx imgsz=640 data=/home/kobe/Desktop/Peppernet/Data/data.yaml  
Visualize:       https://netron.app


'/home/kobe/Desktop/Peppernet/runs/detect/train4/weights/best.onnx'

In [ ]:
from IPython.display import Image as IPyImage
from ultralytics import YOLO
from PIL import Image
import os
import random
import supervision as sv

# Define the folder path
folder_path = '/home/kobe/Desktop/Peppernet/TEST'

# Get a list of all image files in the folder (adjust the file extension if necessary)
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Select a random image file
random_image_path = os.path.join(folder_path, random.choice(image_files))

# Load the image
image = Image.open(random_image_path)

# Load the YOLO model
model = YOLO('/home/kobe/Desktop/Peppernet/runs/detect/train7/weights/best.pt')

# Run the YOLO model on the image
result = model.predict(image, conf=0.375)[0]

# Access detection boxes, confidence, and class labels
result.boxes.xyxy
result.boxes.conf
result.boxes.cls

# Convert detections to supervision format
detections = sv.Detections.from_ultralytics(result)

# Annotate boxes and labels
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)
detections = detections.with_nms(threshold=0.5)  # Adjust threshold as needed
# Create a copy of the image and annotate it
annotated_image = image.copy()
annotated_image = box_annotator.annotate(annotated_image, detections=detections)
annotated_image = label_annotator.annotate(annotated_image, detections=detections)

# Plot the annotated image
# sv.plot_image(annotated_image, size=(10, 10))

output_path = "/home/kobe/Desktop/annotated_image.jpg"
annotated_image.save(output_path)

# Display the saved image using IPython
display(IPyImage(filename=output_path))